## Data cleaning via Python
1 [Life expectancy by state and gender data from Institute for Health Metrics and Evaluation](#IHME)

2 [Life expectancy data by race,gender from Centers for Disease Control](#CDC)

3(i) [Personal income data from Internal Revenue Service](#IRS)

3(ii) [Life expectancy data by income bracket from Health Inequality Project](#HIP)

4 [Population data by state from Census Bureau](#CB)

90% of the data cleaning and formatting are done in Python. Further refining are then performed using Excel PowerPoint due to table download from governmental websites are not in raw format (in structured table format).

### Import libraries

In [1]:
#Import Python Libraries
import numpy as np
from numpy.random import randn
import pandas as pd
# for time series
from pandas import Series, DataFrame
#for scatter plot
from pandas.plotting import scatter_matrix
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import math as m
import seaborn as sns
from pylab import rcParams
import pydot 
import os
import glob

### Creating function to manage memory in this notebook

In [2]:
## Reducing memory
## Function to reduce the DF size
def reduce_memory(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

### Setting data path

In [3]:
os.chdir('C:/Users/vincentlee/Desktop/Data Engineering Platforms/Finalproject')

%pwd

'C:\\Users\\vincentlee\\Desktop\\Data Engineering Platforms\\Finalproject'

### <a name="IHME"></a> 1. Life expectancy by state and gender data from Institute for Health Metrics and Evaluation

To extract life expectancy data by state and gender from 1980-2014

In [4]:
os.chdir('C:/Users/vincentlee/Desktop/Data Engineering Platforms/Finalproject/Life expectancy/IHME_dimgender/statefiles')

%pwd

'C:\\Users\\vincentlee\\Desktop\\Data Engineering Platforms\\Finalproject\\Life expectancy\\IHME_dimgender\\statefiles'

#### Merging 51 dataset (by each state)

Use glob to match the pattern csv files in the folder for merging

In [5]:
extension='csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

# combine all files by rows in the list
merged_IHME = pd.concat([pd.read_csv(f) for f in all_filenames], sort=False)

In [6]:
merged_IHME.shape

(2015415, 15)

In [7]:
merged_IHME.head()

,measure_id,measure_name,location_id,location_name,FIPS,sex_id,sex,age_id,age_name,year_id,metric,val,upper,lower,Variable:
0,26,Life expectancy,523,Alabama,1,1,Male,161,0,1980,Years,68.1195,68.2399,67.9931,NaN
1,26,Life expectancy,523,Alabama,1,1,Male,161,0,1981,Years,68.4319,68.556,68.3121,NaN
2,26,Life expectancy,523,Alabama,1,1,Male,161,0,1982,Years,68.8994,69.0183,68.7775,NaN
3,26,Life expectancy,523,Alabama,1,1,Male,161,0,1983,Years,69.1692,69.2907,69.0512,NaN
4,26,Life expectancy,523,Alabama,1,1,Male,161,0,1984,Years,69.217,69.3322,69.1012,NaN


#### Check missing values

In [8]:
merged_IHME.isnull().sum()

measure_id          3192
measure_name        3192
location_id            0
location_name          0
FIPS                 631
sex_id              3191
sex                 3191
age_id              3188
age_name            3188
year_id             3159
metric              3192
val                 3194
upper               3194
lower               3194
Variable:        2015413
dtype: int64

In [9]:
navalues = pd.isnull(merged_IHME['measure_id'])
merged_IHME_na = merged_IHME[navalues]
merged_IHME_na = reduce_memory(merged_IHME_na)

Mem. usage decreased to  0.39 Mb (0.0% reduction)


In [10]:
#drop NA rows (no measure_id or mesure_name or sex_id) and column Variable
mergedIHME = merged_IHME.dropna(thresh=2)
mergedIHME = mergedIHME.drop('Variable:', axis=1)
mergedIHME.isnull().sum()

measure_id       3192
measure_name     3192
location_id         0
location_name       0
FIPS              631
sex_id           3191
sex              3191
age_id           3188
age_name         3188
year_id          3159
metric           3192
val              3194
upper            3194
lower            3194
dtype: int64

#### Unable to remove rows with NaN - alternatively to sort columns descending

In [11]:
mergedIHME = mergedIHME.sort_values(by=['measure_id', 'measure_name'], ascending=False)
#take out last 3192 rows of null measure id and measure name
mergedIHME = mergedIHME.iloc[1:2012223,]

In [12]:
mergedIHME.isnull().sum()

measure_id         0
measure_name       0
location_id        0
location_name      0
FIPS             631
sex_id             0
sex                0
age_id             0
age_name           0
year_id            0
metric             0
val                2
upper              2
lower              2
dtype: int64

#### Filtering data by state only

In [13]:
states= ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

In [14]:
#create new variable to filter state only from location
mergedIHME['Statevariable'] =  mergedIHME.location_name.isin(states)
mergedIHME.head()

,measure_id,measure_name,location_id,location_name,FIPS,sex_id,sex,age_id,age_name,year_id,metric,val,upper,lower,Statevariable
2,34,Mortality risk,523,Alabama,1,2,Female,161,0,1981,Percentage,NaN,NaN,NaN,True
1,26,Life expectancy,102,United States,NaN,1,Male,1,Under 5,1980,Years,NaN,NaN,NaN,False
7140,34,Mortality risk,523,Alabama,1,1,Male,1,Under 5,1980,Percentage,2.13013,2.16987,2.09148,True
7141,34,Mortality risk,523,Alabama,1,1,Male,1,Under 5,1981,Percentage,2.03719,2.07385,2.00197,True
7142,34,Mortality risk,523,Alabama,1,1,Male,1,Under 5,1982,Percentage,1.94007,1.97464,1.90486,True


In [15]:
#filter only the states
mergedIHME2 = mergedIHME[mergedIHME.Statevariable == True]
mergedIHME2.head()

,measure_id,measure_name,location_id,location_name,FIPS,sex_id,sex,age_id,age_name,year_id,metric,val,upper,lower,Statevariable
2,34,Mortality risk,523,Alabama,1,2,Female,161,0,1981,Percentage,NaN,NaN,NaN,True
7140,34,Mortality risk,523,Alabama,1,1,Male,1,Under 5,1980,Percentage,2.13013,2.16987,2.09148,True
7141,34,Mortality risk,523,Alabama,1,1,Male,1,Under 5,1981,Percentage,2.03719,2.07385,2.00197,True
7142,34,Mortality risk,523,Alabama,1,1,Male,1,Under 5,1982,Percentage,1.94007,1.97464,1.90486,True
7143,34,Mortality risk,523,Alabama,1,1,Male,1,Under 5,1983,Percentage,1.84181,1.87497,1.8093,True


In [17]:
#filter only male and female
mergedIHME2 = mergedIHME[mergedIHME.sex != 'Both']
mergedIHME2.head()

,measure_id,measure_name,location_id,location_name,FIPS,sex_id,sex,age_id,age_name,year_id,metric,val,upper,lower,Statevariable
2,34,Mortality risk,523,Alabama,1,2,Female,161,0,1981,Percentage,NaN,NaN,NaN,True
1,26,Life expectancy,102,United States,NaN,1,Male,1,Under 5,1980,Years,NaN,NaN,NaN,False
7140,34,Mortality risk,523,Alabama,1,1,Male,1,Under 5,1980,Percentage,2.13013,2.16987,2.09148,True
7141,34,Mortality risk,523,Alabama,1,1,Male,1,Under 5,1981,Percentage,2.03719,2.07385,2.00197,True
7142,34,Mortality risk,523,Alabama,1,1,Male,1,Under 5,1982,Percentage,1.94007,1.97464,1.90486,True


In [18]:
#transform data for the SQL schema
#remove columns that won't be used e.g. upper, lower, state variable, age, measure_id

mergedIHME2 = mergedIHME2.drop(['measure_id','location_id', 'FIPS','sex_id','age_id','metric',
                                'age_name','upper','lower','Statevariable',], axis=1)
mergedIHME2.head()

,measure_name,location_name,sex,year_id,val
2,Mortality risk,Alabama,Female,1981,NaN
1,Life expectancy,United States,Male,1980,NaN
7140,Mortality risk,Alabama,Male,1980,2.13013
7141,Mortality risk,Alabama,Male,1981,2.03719
7142,Mortality risk,Alabama,Male,1982,1.94007


#### Create IDs for state and gender

In [19]:
#get state ids for the states
state_codes = {
    'Washington': '53', 'Delaware': '10', 'District of Columbia': '11', 
    'Wisconsin': '55', 'West Virginia': '54', 'Hawaii': '15',
    'Florida': '12', 'Wyoming': '56', 'Puerto Rico': '72', 
    'New Jersey': '34', 'New Mexico': '35', 'Texas': '48',
    'Louisiana': '22', 'North Carolina': '37', 'North Dakota': '38', 
    'Nebraska': '31', 'Tennessee': '47', 'New York': '36',
    'Pennsylvania': '42', 'Alaska': '2', 'Nevada': '32', 'New Hampshire': '33', 
    'Virginia': '51', 'Colorado': '8','California': '6', 'Alabama': '1', 'Arkansas': '5', 
    'Vermont': '50', 'Illinois': '17', 'Georgia': '13',
    'Indiana': '18', 'Iowa': '19', 'Massachusetts': '25', 
    'Arizona': '04', 'Idaho': '16', 'Connecticut': '9',
    'Maine': '23', 'Maryland': '24', 'Oklahoma': '40', 
    'Ohio': '39', 'Utah': '49', 'Missouri': '29',
    'Minnesota': '27', 'Michigan': '26', 'Rhode Island': '44', 
    'Kansas': '20', 'Montana': '30', 'Mississippi': '28',
    'South Carolina': '45', 'Kentucky': '21', 'Oregon': '41', 'South Dakota': '46'
}

In [20]:
'''strip leading and trailing space for state'''
mergedIHME2['location_name'] = mergedIHME2['location_name'].str.strip()
mergedIHME2['stateid'] = mergedIHME2.location_name.map(state_codes)
mergedIHME2.tail()

,measure_name,location_name,sex,year_id,val,stateid
2480,Life expectancy,Weston County,Female,2010,81.1764,NaN
2481,Life expectancy,Weston County,Female,2011,81.1927,NaN
2482,Life expectancy,Weston County,Female,2012,81.1981,NaN
2483,Life expectancy,Weston County,Female,2013,81.311,NaN
2484,Life expectancy,Weston County,Female,2014,81.236,NaN


In [21]:
mergedIHME2.shape

(1341482, 6)

In [22]:
# drop all the county's data from location_name
mergedIHME2 = mergedIHME2.dropna()
mergedIHME2.isnull().sum()

measure_name     0
location_name    0
sex              0
year_id          0
val              0
stateid          0
dtype: int64

In [23]:
mergedIHME2.shape

(21840, 6)

In [26]:
#get state ids for the states
gender_id = {'Female': '1', 'Male': '2'}

In [27]:
mergedIHME2['genderid'] = mergedIHME2.sex.map(gender_id)
mergedIHME2.tail()

,measure_name,location_name,sex,year_id,val,stateid,genderid
65,Life expectancy,Wyoming,Female,2010,80.5838,56,1
66,Life expectancy,Wyoming,Female,2011,80.6387,56,1
67,Life expectancy,Wyoming,Female,2012,80.6883,56,1
68,Life expectancy,Wyoming,Female,2013,80.7136,56,1
69,Life expectancy,Wyoming,Female,2014,80.6826,56,1


In [28]:
#rename the columns
mergedIHME2.columns = ['measure_type','state','gendername','year','value','stateid','genderid']
mergedIHME2.head()

,measure_type,state,gendername,year,value,stateid,genderid
7140,Mortality risk,Alabama,Male,1980,2.13013,1,2
7141,Mortality risk,Alabama,Male,1981,2.03719,1,2
7142,Mortality risk,Alabama,Male,1982,1.94007,1,2
7143,Mortality risk,Alabama,Male,1983,1.84181,1,2
7144,Mortality risk,Alabama,Male,1984,1.77745,1,2


In [29]:
#reorder the columns
mergedIHME2 = mergedIHME2[['genderid','gendername','measure_type','value','year','stateid']]
mergedIHME2.head()

,genderid,gendername,measure_type,value,year,stateid
7140,2,Male,Mortality risk,2.13013,1980,1
7141,2,Male,Mortality risk,2.03719,1981,1
7142,2,Male,Mortality risk,1.94007,1982,1
7143,2,Male,Mortality risk,1.84181,1983,1
7144,2,Male,Mortality risk,1.77745,1984,1


In [30]:
#split dataset by measure type in order to have two columns: one value for life expectancy/
# Another column for value of mortality
IHME_lifeexpectancy = mergedIHME2[mergedIHME2.measure_type == "Life expectancy"]
IHME_mortality = mergedIHME2[mergedIHME2.measure_type == "Mortality risk"]

In [31]:
IHME_lifeexpectancy.head()

,genderid,gendername,measure_type,value,year,stateid
0,2,Male,Life expectancy,68.1195,1980,1
1,2,Male,Life expectancy,68.4319,1981,1
2,2,Male,Life expectancy,68.8994,1982,1
3,2,Male,Life expectancy,69.1692,1983,1
4,2,Male,Life expectancy,69.217,1984,1


In [32]:
IHME_mortality.columns = ['genderid','gendername','measure_type2','mortality','year','stateid']
IHME_mortality.head()

,genderid,gendername,measure_type2,mortality,year,stateid
7140,2,Male,Mortality risk,2.13013,1980,1
7141,2,Male,Mortality risk,2.03719,1981,1
7142,2,Male,Mortality risk,1.94007,1982,1
7143,2,Male,Mortality risk,1.84181,1983,1
7144,2,Male,Mortality risk,1.77745,1984,1


In [33]:
# inner join them
merged_IHME3 = pd.merge(left=IHME_lifeexpectancy,right=IHME_mortality,left_on='stateid', 
                        right_on='stateid')
merged_IHME3.head()

,genderid_x,gendername_x,measure_type,value,year_x,stateid,genderid_y,gendername_y,measure_type2,mortality,year_y
0,2,Male,Life expectancy,68.1195,1980,1,2,Male,Mortality risk,2.13013,1980
1,2,Male,Life expectancy,68.1195,1980,1,2,Male,Mortality risk,2.03719,1981
2,2,Male,Life expectancy,68.1195,1980,1,2,Male,Mortality risk,1.94007,1982
3,2,Male,Life expectancy,68.1195,1980,1,2,Male,Mortality risk,1.84181,1983
4,2,Male,Life expectancy,68.1195,1980,1,2,Male,Mortality risk,1.77745,1984


In [34]:
mergedIHME3 = merged_IHME3.drop(['measure_type','genderid_y','gendername_y', 
                                'measure_type2','year_y'],axis=1)
mergedIHME3.head()

,genderid_x,gendername_x,value,year_x,stateid,mortality
0,2,Male,68.1195,1980,1,2.13013
1,2,Male,68.1195,1980,1,2.03719
2,2,Male,68.1195,1980,1,1.94007
3,2,Male,68.1195,1980,1,1.84181
4,2,Male,68.1195,1980,1,1.77745


In [35]:
#reorder columns and rename
mergedIHME3 = mergedIHME3[['genderid_x','gendername_x','value','mortality','year_x','stateid']]
mergedIHME3.columns = ['genderid','gendername','gender_lifeexp','mortality','year','stateid']
mergedIHME3.head()

,genderid,gendername,gender_lifeexp,mortality,year,stateid
0,2,Male,68.1195,2.13013,1980,1
1,2,Male,68.1195,2.03719,1980,1
2,2,Male,68.1195,1.94007,1980,1
3,2,Male,68.1195,1.84181,1980,1
4,2,Male,68.1195,1.77745,1980,1


#### Save merged files to CSV

In [36]:
# export to csv
mergedIHME3.to_csv( "combinedstate_lifeexp_IMHE.csv", index=False, encoding='utf-8-sig')

### <a name="CDC"></a> 2. Life expectancy data by race,gender from Centers for Disease Control

To extract life expectancy by sex and race data from 1900-2017

In [37]:
os.chdir('C:/Users/vincentlee/Desktop/Data Engineering Platforms/Finalproject/Life expectancy/CDC_dimrace')

%pwd

'C:\\Users\\vincentlee\\Desktop\\Data Engineering Platforms\\Finalproject\\Life expectancy\\CDC_dimrace'

Reading file and exploration of data

In [38]:
CDC_lifeexp = pd.read_excel('CDClifeexpectancy.xlsx')
CDC_lifeexp.head(10)

,"Table 4. Life expectancy at birth, at age 65, and at age 75, by sex, race, and Hispanic origin: United States, selected years 1900–2017",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Excel version (with more data years and standa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[Data are based on death certificates],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,All races,All races,All races,White,White,White,Black,Black,Black,Hispanics,Hispanics,Hispanics
3,Specified age and year,Both sexes,Male,Female,Both sexes,Male,Female,Both sexes,Male,Female,Both sexes,Male,Female
4,At birth,Life expectancy in years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"19003/,4/",47.3,46.3,48.3,47.6,46.6,48.7,33,32.5,33.5,NaN,NaN,NaN
6,"19503/,4/",68.2,65.6,71.1,69.1,66.5,72.2,60.8,59.1,62.9,NaN,NaN,NaN
7,19604/,69.7,66.6,73.1,70.6,67.4,74.1,63.6,61.1,66.3,NaN,NaN,NaN
8,1970,70.8,67.1,74.7,71.7,68,75.6,64.1,60,68.3,NaN,NaN,NaN
9,1975,72.6,68.8,76.6,73.4,69.5,77.3,66.8,62.4,71.3,NaN,NaN,NaN


In [39]:
CDC_lifeexp.tail()

,"Table 4. Life expectancy at birth, at age 65, and at age 75, by sex, race, and Hispanic origin: United States, selected years 1900–2017",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,- - - Data not available.\r\n1/Life expectancy...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,NOTES: Populations for computing life expectan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,"SOURCE: NCHS, National Vital Statistics System...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Data is placed in a formatted table so we need to clean it up

In [40]:
CDC_lifeexp.columns= ['Year','lifeexp_all','lifeexp_all_male','lifeexp_all_female',
                     'lifeexp_white_all','lifeexp_white_male','lifeexp_white_female',
                     'lifeexp_black_all','lifeexp_black_male','lifeexp_black_female',
                     'lifeexp_hispanics_all','lifeexp_hispanics_male','lifeexp_hispanics_female']
#filter text at top and bottom of table
CDC_lifeexp = CDC_lifeexp.loc[5:130,]
CDC_lifeexp.head()

,Year,lifeexp_all,lifeexp_all_male,lifeexp_all_female,lifeexp_white_all,lifeexp_white_male,lifeexp_white_female,lifeexp_black_all,lifeexp_black_male,lifeexp_black_female,lifeexp_hispanics_all,lifeexp_hispanics_male,lifeexp_hispanics_female
5,"19003/,4/",47.3,46.3,48.3,47.6,46.6,48.7,33,32.5,33.5,NaN,NaN,NaN
6,"19503/,4/",68.2,65.6,71.1,69.1,66.5,72.2,60.8,59.1,62.9,NaN,NaN,NaN
7,19604/,69.7,66.6,73.1,70.6,67.4,74.1,63.6,61.1,66.3,NaN,NaN,NaN
8,1970,70.8,67.1,74.7,71.7,68,75.6,64.1,60,68.3,NaN,NaN,NaN
9,1975,72.6,68.8,76.6,73.4,69.5,77.3,66.8,62.4,71.3,NaN,NaN,NaN


In [41]:
CDC_lifeexp.isnull().sum()

Year                         1
lifeexp_all                  3
lifeexp_all_male             3
lifeexp_all_female           3
lifeexp_white_all            3
lifeexp_white_male           3
lifeexp_white_female         3
lifeexp_black_all            3
lifeexp_black_male           3
lifeexp_black_female         3
lifeexp_hispanics_all       90
lifeexp_hispanics_male      90
lifeexp_hispanics_female    90
dtype: int64

In [44]:
#remove invalid years
CDC_lifeexp.iloc[-5:,0]

122    2010
123    2011
124    2012
125    2013
126    2014
Name: Year, dtype: object

In [45]:
#subset data for year 2006-2017 only
year = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']
CDC_lifeexp= CDC_lifeexp[CDC_lifeexp.Year.isin(year)]
CDC_lifeexp.head()

,Year,lifeexp_all,lifeexp_all_male,lifeexp_all_female,lifeexp_white_all,lifeexp_white_male,lifeexp_white_female,lifeexp_black_all,lifeexp_black_male,lifeexp_black_female,lifeexp_hispanics_all,lifeexp_hispanics_male,lifeexp_hispanics_female
36,2006,77.8,75.2,80.3,78.2,75.7,80.6,73.1,69.5,76.4,80.3,77.5,82.9
37,2007,78.1,75.5,80.6,78.4,75.9,80.8,73.5,69.9,76.7,80.7,77.8,83.2
38,2008,78.2,75.6,80.6,78.4,76,80.7,73.9,70.5,77,80.8,78,83.3
39,2009,78.5,76,80.9,78.7,76.3,81,74.4,71,77.4,81.1,78.4,83.5
40,2010,78.7,76.2,81,78.8,76.4,81.1,74.7,71.5,77.7,81.7,78.8,84.3


In [46]:
CDC_lifeexp.isnull().sum()

Year                        0
lifeexp_all                 0
lifeexp_all_male            0
lifeexp_all_female          0
lifeexp_white_all           0
lifeexp_white_male          0
lifeexp_white_female        0
lifeexp_black_all           0
lifeexp_black_male          0
lifeexp_black_female        0
lifeexp_hispanics_all       0
lifeexp_hispanics_male      0
lifeexp_hispanics_female    0
dtype: int64

To further format the data, the table is saved into csv for pivoting in Excel

In [47]:
# export to csv
CDC_lifeexp.to_csv( "cleaned_lifeexpbyrace_CDC.csv", index=False, encoding='utf-8-sig')

### <a name="IRS"></a>3(i). Personal income data from Internal Revenue Service

In [48]:
os.chdir('C:/Users/vincentlee/Desktop/Data Engineering Platforms/Finalproject/Income data_dimincome/IRS/rawfiles')

%pwd

'C:\\Users\\vincentlee\\Desktop\\Data Engineering Platforms\\Finalproject\\Income data_dimincome\\IRS\\rawfiles'

Merged all the yearly files into one file

In [51]:
extension='csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

# combine all files by rows in the list
merged_IRS_20122017 = pd.concat([pd.read_csv(f) for f in all_filenames], sort=False)

In [52]:
merged_IRS_20122017.shape

(3498, 247)

In [53]:
merged_IRS_20122017.head()

,year,State,agi_stub,N1,MARS1,MARS2,MARS4,PREP,N2,NUMDEP,...,A21020,ELF,CPREP,DIR_DEP,N20950,A20950,N11900,A11900,N12000,A12000
0,2012,US,0.0,"145,025,450","66,769,990","53,700,420","21,722,220","80,992,600","287,692,220","97,957,080",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,US,1.0,"2,248,050","1,381,410","674,380","101,280","1,510,430","3,346,940","641,240",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,US,2.0,"22,638,580","18,078,240","1,850,490","2,408,590","11,166,520","23,004,000","5,663,120",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,US,3.0,"34,059,840","18,978,550","5,772,170","8,714,590","18,632,790","61,847,430","23,501,240",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012,US,4.0,"33,965,310","16,169,080","9,978,030","6,847,880","18,453,940","68,387,910","24,554,380",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
#remove the irrelevant columns
#just need the columns associated with number of returns by size of adjusted gross income

merged_IRS_20122017= merged_IRS_20122017.iloc[:,:7]
merged_IRS_20122017.columns=['year','state','income_bracket','totalnoreturns','noofsinglereturns',
                             'noofjointreturns','noofheadofhouseholdreturns']
merged_IRS_20122017.head()

,year,state,income_bracket,totalnoreturns,noofsinglereturns,noofjointreturns,noofheadofhouseholdreturns
0,2012,US,0.0,"145,025,450","66,769,990","53,700,420","21,722,220"
1,2012,US,1.0,"2,248,050","1,381,410","674,380","101,280"
2,2012,US,2.0,"22,638,580","18,078,240","1,850,490","2,408,590"
3,2012,US,3.0,"34,059,840","18,978,550","5,772,170","8,714,590"
4,2012,US,4.0,"33,965,310","16,169,080","9,978,030","6,847,880"


In [55]:
#remove US from state
merged_IRS_20122017 = merged_IRS_20122017[merged_IRS_20122017.state != 'US']
merged_IRS_20122017.head()

,year,state,income_bracket,totalnoreturns,noofsinglereturns,noofjointreturns,noofheadofhouseholdreturns
11,2012,AL,0.0,"2,050,890","810,470","774,590","427,690"
12,2012,AL,1.0,"25,810","14,500","9,170","1,150"
13,2012,AL,2.0,"315,080","231,420","28,860","50,770"
14,2012,AL,3.0,"575,210","259,590","100,250","206,200"
15,2012,AL,4.0,"491,320","194,420","159,240","123,380"


In [56]:
merged_IRS_20122017.shape

(3476, 7)

In [58]:
IRS_20122017_2 = merged_IRS_20122017.drop(['noofsinglereturns','noofjointreturns',
                                                'noofheadofhouseholdreturns'],axis=1)
IRS_20122017_2 = IRS_20122017_2.sort_values(by=['year','state','income_bracket'])
IRS_20122017_2.head()

,year,state,income_bracket,totalnoreturns
22,2012,AK,0.0,"363,090"
23,2012,AK,1.0,"3,650"
24,2012,AK,2.0,"59,510"
25,2012,AK,3.0,"69,490"
26,2012,AK,4.0,"80,900"


#### Merge IRS data for 2012-2017 with data for 2008-2011

In [59]:
IRS_20122017_2.columns=['year','state','incomeid','population']
IRS_20122017_2.head()

,year,state,incomeid,population
22,2012,AK,0.0,"363,090"
23,2012,AK,1.0,"3,650"
24,2012,AK,2.0,"59,510"
25,2012,AK,3.0,"69,490"
26,2012,AK,4.0,"80,900"


In [60]:
#map state abbreviation to full state name

state_abbtofull = {
        'AK': 'Alaska','AL': 'Alabama','AR': 'Arkansas',
        'AS': 'American Samoa','AZ': 'Arizona',
        'CA': 'California', 'CO': 'Colorado',
        'CT': 'Connecticut','DC': 'District of Columbia',
        'DE': 'Delaware','FL': 'Florida',
        'GA': 'Georgia','GU': 'Guam',
        'HI': 'Hawaii','IA': 'Iowa',
        'ID': 'Idaho','IL': 'Illinois',
        'IN': 'Indiana','KS': 'Kansas',
        'KY': 'Kentucky','LA': 'Louisiana',
        'MA': 'Massachusetts','MD': 'Maryland',
        'ME': 'Maine','MI': 'Michigan',
        'MN': 'Minnesota','MO': 'Missouri',
        'MP': 'Northern Mariana Islands','MS': 'Mississippi',
        'MT': 'Montana','NA': 'National',
        'NC': 'North Carolina','ND': 'North Dakota',
        'NE': 'Nebraska','NH': 'New Hampshire',
        'NJ': 'New Jersey','NM': 'New Mexico',
        'NV': 'Nevada','NY': 'New York',
        'OH': 'Ohio','OK': 'Oklahoma',
        'OR': 'Oregon','PA': 'Pennsylvania',
        'PR': 'Puerto Rico','RI': 'Rhode Island',
        'SC': 'South Carolina','SD': 'South Dakota',
        'TN': 'Tennessee','TX': 'Texas',
        'UT': 'Utah','VA': 'Virginia',
        'VI': 'Virgin Islands','VT': 'Vermont',
        'WA': 'Washington','WI': 'Wisconsin',
        'WV': 'West Virginia','WY': 'Wyoming'
}

IRS_20122017_2.state = IRS_20122017_2.state.map(state_abbtofull)
IRS_20122017_2= IRS_20122017_2.dropna()  #drop unknown states i.e. OA and US
IRS_20122017_2.isnull().sum()

year          0
state         0
incomeid      0
population    0
dtype: int64

In [61]:
IRS_20122017_2.shape

(1122, 4)

In [63]:
# merged IRS 2008 from 2011 was in a formatted table format by each year
# data cleaning and formatting was done in Excel for year 2008 to 2011

IRS_20082011 = pd.read_csv('merged_IRS_20082011_cleaned.csv')
IRS_20082011.columns=['year','state','incomeid','population']
IRS_20082011.head()

,year,state,incomeid,population
0,2008,Alabama,1,NaN
1,2008,Alabama,2,1458690.0
2,2008,Alabama,3,259775.0
3,2008,Alabama,4,154949.0
4,2008,Alaska,1,NaN


In [64]:
merged_IRSdata = pd.concat([IRS_20122017_2,IRS_20082011])
merged_IRSdata.isnull().sum()

year            0
state           0
incomeid        0
population    102
dtype: int64

Create stateid for the merged IRS data

In [65]:
state_codes = {
    'Washington': '53', 'Delaware': '10', 'District of Columbia': '11', 
    'Wisconsin': '55', 'West Virginia': '54', 'Hawaii': '15',
    'Florida': '12', 'Wyoming': '56', 'Puerto Rico': '72', 
    'New Jersey': '34', 'New Mexico': '35', 'Texas': '48',
    'Louisiana': '22', 'North Carolina': '37', 'North Dakota': '38', 
    'Nebraska': '31', 'Tennessee': '47', 'New York': '36',
    'Pennsylvania': '42', 'Alaska': '2', 'Nevada': '32', 'New Hampshire': '33', 
    'Virginia': '51', 'Colorado': '8','California': '6', 'Alabama': '1', 'Arkansas': '5', 
    'Vermont': '50', 'Illinois': '17', 'Georgia': '13',
    'Indiana': '18', 'Iowa': '19', 'Massachusetts': '25', 
    'Arizona': '04', 'Idaho': '16', 'Connecticut': '9',
    'Maine': '23', 'Maryland': '24', 'Oklahoma': '40', 
    'Ohio': '39', 'Utah': '49', 'Missouri': '29',
    'Minnesota': '27', 'Michigan': '26', 'Rhode Island': '44', 
    'Kansas': '20', 'Montana': '30', 'Mississippi': '28',
    'South Carolina': '45', 'Kentucky': '21', 'Oregon': '41', 'South Dakota': '46'
}

In [66]:
'''strip leading and trailing space for state'''
merged_IRSdata['state'] = merged_IRSdata['state'].str.strip()
merged_IRSdata['stateid'] = merged_IRSdata.state.map(state_codes)
merged_IRSdata.isnull().sum()

year            0
state           0
incomeid        0
population    102
stateid         0
dtype: int64

In [67]:
merged_IRSdata.head()

,year,state,incomeid,population,stateid
22,2012,Alaska,0.0,"363,090",2
23,2012,Alaska,1.0,"3,650",2
24,2012,Alaska,2.0,"59,510",2
25,2012,Alaska,3.0,"69,490",2
26,2012,Alaska,4.0,"80,900",2


Export table to csv for merging dataset with life expectancy data by income bracket

In [68]:
# export to csv
merged_IRSdata.to_csv( "merged_IRS_20082017.csv", index=False, encoding='utf-8-sig')

### <a name="HIP"></a>3(ii). Life expectancy data by income bracket from Health Inequality Project

In [69]:
os.chdir('C:/Users/vincentlee/Desktop/Data Engineering Platforms/Finalproject/Income data_dimincome/Health inequality project')

%pwd

'C:\\Users\\vincentlee\\Desktop\\Data Engineering Platforms\\Finalproject\\Income data_dimincome\\Health inequality project'

In [70]:
lifeexp_income = pd.read_csv('health_ineq_online_table_5.csv')

In [71]:
lifeexp_income= lifeexp_income[['statename','year','le_raceadj_q1_F',
               'le_raceadj_q2_F', 'le_raceadj_q3_F',
               'le_raceadj_q4_F','le_raceadj_q1_M',
               'le_raceadj_q2_M','le_raceadj_q3_M','le_raceadj_q4_M']]
lifeexp_income.columns = ['state','year','lifeexpq1_F','lifeexpq2_F',
                         'lifeexpq3_F','lifeexpq4_F','lifeexpq1_M','lifeexpq2_M',
                         'lifeexpq3_M','lifeexpq4_M']
lifeexp_income.head()

,state,year,lifeexpq1_F,lifeexpq2_F,lifeexpq3_F,lifeexpq4_F,lifeexpq1_M,lifeexpq2_M,lifeexpq3_M,lifeexpq4_M
0,Alabama,2001,81.736870,84.270462,85.575615,86.917931,74.852470,80.159592,83.565346,84.868271
1,Alabama,2002,82.294128,85.626312,87.244286,86.241699,74.579124,78.600525,83.666740,84.714966
2,Alabama,2003,81.446388,84.912849,84.798073,87.874893,76.438599,80.262939,83.823204,83.966675
3,Alabama,2004,83.548531,83.739426,86.119102,87.538551,74.749626,80.289833,82.965218,87.103699
4,Alabama,2005,82.799507,85.402199,86.473816,87.898590,75.539764,80.295036,82.564537,85.346405


In [72]:
state_codes2 = {
    'Washington': '53', 'Delaware': '10', 'District Of Columbia': '11', 
    'Wisconsin': '55', 'West Virginia': '54', 'Hawaii': '15',
    'Florida': '12', 'Wyoming': '56', 'Puerto Rico': '72', 
    'New Jersey': '34', 'New Mexico': '35', 'Texas': '48',
    'Louisiana': '22', 'North Carolina': '37', 'North Dakota': '38', 
    'Nebraska': '31', 'Tennessee': '47', 'New York': '36',
    'Pennsylvania': '42', 'Alaska': '2', 'Nevada': '32', 'New Hampshire': '33', 
    'Virginia': '51', 'Colorado': '8','California': '6', 'Alabama': '1', 'Arkansas': '5', 
    'Vermont': '50', 'Illinois': '17', 'Georgia': '13',
    'Indiana': '18', 'Iowa': '19', 'Massachusetts': '25', 
    'Arizona': '04', 'Idaho': '16', 'Connecticut': '9',
    'Maine': '23', 'Maryland': '24', 'Oklahoma': '40', 
    'Ohio': '39', 'Utah': '49', 'Missouri': '29',
    'Minnesota': '27', 'Michigan': '26', 'Rhode Island': '44', 
    'Kansas': '20', 'Montana': '30', 'Mississippi': '28',
    'South Carolina': '45', 'Kentucky': '21', 'Oregon': '41', 'South Dakota': '46'
}

In [73]:
'''strip leading and trailing space for state'''
lifeexp_income['state'] = lifeexp_income['state'].str.strip()
lifeexp_income['stateid'] = lifeexp_income.state.map(state_codes2)
lifeexp_income.isnull().sum()

state          0
year           0
lifeexpq1_F    0
lifeexpq2_F    0
lifeexpq3_F    0
lifeexpq4_F    0
lifeexpq1_M    0
lifeexpq2_M    0
lifeexpq3_M    0
lifeexpq4_M    0
stateid        0
dtype: int64

In [74]:
lifeexp_income.head()

,state,year,lifeexpq1_F,lifeexpq2_F,lifeexpq3_F,lifeexpq4_F,lifeexpq1_M,lifeexpq2_M,lifeexpq3_M,lifeexpq4_M,stateid
0,Alabama,2001,81.736870,84.270462,85.575615,86.917931,74.852470,80.159592,83.565346,84.868271,1
1,Alabama,2002,82.294128,85.626312,87.244286,86.241699,74.579124,78.600525,83.666740,84.714966,1
2,Alabama,2003,81.446388,84.912849,84.798073,87.874893,76.438599,80.262939,83.823204,83.966675,1
3,Alabama,2004,83.548531,83.739426,86.119102,87.538551,74.749626,80.289833,82.965218,87.103699,1
4,Alabama,2005,82.799507,85.402199,86.473816,87.898590,75.539764,80.295036,82.564537,85.346405,1


Export table to csv for merging with IRS income data

In [75]:
# export to csv
lifeexp_income.to_csv( "lifeexp_income.csv", index=False, encoding='utf-8-sig')

### <a name="CB"></a>4. Population data by state from Census Bureau

In [76]:
os.chdir('C:/Users/vincentlee/Desktop/Data Engineering Platforms/Finalproject/CensusBureau')

%pwd

'C:\\Users\\vincentlee\\Desktop\\Data Engineering Platforms\\Finalproject\\CensusBureau'

In [77]:
def popstateDF(dataset):
    x= pd.read_excel(dataset)
    x= x.iloc[3:75,:13]
    x.columns=['State','SexRace','Apr2000','2000','2001',
               '2002','2003','2004','2005','2006','2007','2008','2009']
    return x

Combine all state level data on population

In [78]:
state1=popstateDF('st-est00int-03-01.xls')
state2=popstateDF('st-est00int-03-02.xls')
state4=popstateDF('st-est00int-03-04.xls')
state5=popstateDF('st-est00int-03-05.xls')
state6=popstateDF('st-est00int-03-06.xls')
state8=popstateDF('st-est00int-03-08.xls')
state9=popstateDF('st-est00int-03-09.xls')
state10=popstateDF('st-est00int-03-10.xls')
state11=popstateDF('st-est00int-03-11.xls')
state12=popstateDF('st-est00int-03-12.xls')
state13=popstateDF('st-est00int-03-13.xls')
state15=popstateDF('st-est00int-03-15.xls')
state16=popstateDF('st-est00int-03-16.xls')
state17=popstateDF('st-est00int-03-17.xls')
state18=popstateDF('st-est00int-03-18.xls')
state19=popstateDF('st-est00int-03-19.xls')
state20=popstateDF('st-est00int-03-20.xls')
state21=popstateDF('st-est00int-03-21.xls')
state22=popstateDF('st-est00int-03-22.xls')
state23=popstateDF('st-est00int-03-23.xls')
state24=popstateDF('st-est00int-03-24.xls')
state25=popstateDF('st-est00int-03-25.xls')
state26=popstateDF('st-est00int-03-26.xls')
state27=popstateDF('st-est00int-03-27.xls')
state28=popstateDF('st-est00int-03-28.xls')
state29=popstateDF('st-est00int-03-29.xls')
state30=popstateDF('st-est00int-03-30.xls')
state31=popstateDF('st-est00int-03-31.xls')
state32=popstateDF('st-est00int-03-32.xls')
state33=popstateDF('st-est00int-03-33.xls')
state34=popstateDF('st-est00int-03-34.xls')
state35=popstateDF('st-est00int-03-35.xls')
state36=popstateDF('st-est00int-03-36.xls')
state37=popstateDF('st-est00int-03-37.xls')
state38=popstateDF('st-est00int-03-38.xls')
state39=popstateDF('st-est00int-03-39.xls')
state40=popstateDF('st-est00int-03-40.xls')
state41=popstateDF('st-est00int-03-41.xls')
state42=popstateDF('st-est00int-03-42.xls')
state44=popstateDF('st-est00int-03-44.xls')
state45=popstateDF('st-est00int-03-45.xls')
state46=popstateDF('st-est00int-03-46.xls')
state47=popstateDF('st-est00int-03-47.xls')
state48=popstateDF('st-est00int-03-48.xls')
state49=popstateDF('st-est00int-03-49.xls')
state50=popstateDF('st-est00int-03-50.xls')
state51=popstateDF('st-est00int-03-51.xls')
state53=popstateDF('st-est00int-03-53.xls')
state54=popstateDF('st-est00int-03-54.xls')
state55=popstateDF('st-est00int-03-55.xls')
state56=popstateDF('st-est00int-03-56.xls')

In [79]:
popstate_df = pd.concat([state1,state10,state11,state12,state13,
                         state15,state16,state17,state18,state19,
                         state2,state20,state21,state22,state23,
                         state24,state25,state26,state27,state28,
                         state29,state30,state31,state32,state33,
                         state34,state35,state36,state37,state38,
                         state39,state4,state40,state41,state42,
                         state44,state45,state46,state47,state48,
                         state49,state5,state50,state51,state53,
                         state54,state55,state56,state6,state8,state9])

In [80]:
popstate_df.shape

(3672, 13)

In [81]:
popstate_df.isnull().sum()

State      0
SexRace    0
Apr2000    0
2000       0
2001       0
2002       0
2003       0
2004       0
2005       0
2006       0
2007       0
2008       0
2009       0
dtype: int64

In [82]:
popstate_df= popstate_df.drop('Apr2000',axis=1)
popstate_df.head()

,State,SexRace,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
3,Alabama,BOTH SEXES,4452173,4467634.0,4480089.0,4503491.0,4530729.0,4569805.0,4628981.0,4672840.0,4718206.0,4757938.0
4,Alabama,.One Race,4415743,4428808.0,4438999.0,4459951.0,4484597.0,4520807.0,4576943.0,4617741.0,4659602.0,4695919.0
5,Alabama,..White,3196875,3201387.0,3204279.0,3215079.0,3227772.0,3249443.0,3283056.0,3308005.0,3332141.0,3353244.0
6,Alabama,..Black,1161454,1167403.0,1171672.0,1178398.0,1186375.0,1197062.0,1215481.0,1227620.0,1241613.0,1252307.0
7,Alabama,..AIAN,23262,24084.0,24966.0,25920.0,26755.0,27606.0,28666.0,29666.0,30722.0,31825.0


In [83]:
# export to csv
popstate_df.to_csv( "popest20002009.csv", index=False, encoding='utf-8-sig')

Population data will be merged with hospital, expenditure, state_lifeexp and uninsured data via Excel pivot.